<a href="https://colab.research.google.com/github/dowrave/Data_Analysis_Projects/blob/main/Dacon_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from pandas import CategoricalDtype
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv')

In [ ]:
train.head(2)

In [ ]:
train.info()

In [ ]:
train.head(4)

# 데이터 탐색

In [ ]:
train.hist(bins = 20, figsize = (20, 20))

### 질적 데이터

- 전체 데이터 중 유망주의 비율 : $0.361$

In [ ]:
train['Prospect'].sum() / train['Prospect'].count()

#### 1. `PreferredFoot, WorkRate`
선요약 )  
- 특히 `WorkRate`의 경우는 높을수록 좋을 것이라는 생각을 하고 접근했지만, 가장 좋은 수치는 `Medium, Medium`에서 나왔음
- `PreferredFoot`의 경우 또한 `Left`에서 희소성이 있을 거라 생각하고 접근했지만, 그렇지는 않았음
- 둘 다 특별히 고칠 요소는 없는 것으로 보임


In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))

fig.suptitle("Preferred Foot : Total vs Prospect")
(train['PreferredFoot'].value_counts()
                        .to_frame()
                        .T
                        .plot(kind = 'bar', 
                              stacked = True, 
                              rot = 0,
                              ax = ax[0])
                        )
ax[0].legend().set_visible(False)

(train[train['Prospect'] == 1]['PreferredFoot'].value_counts()
                                                .to_frame()
                                                .T
                                                .plot(kind = 'bar', 
                                                      stacked = True, 
                                                      rot = 0,
                                                      ax = ax[1],
))

ax[1].set_ylim(ax[0].get_ylim())

In [ ]:
# 위 그래프는 이렇게 그려도 되지 않을까?
sns.set_style('whitegrid')
plt.figure(figsize = (10, 10))
grid = sns.FacetGrid(data = train, col = 'Prospect')
grid.map(sns.countplot, 'PreferredFoot', palette= 'tab10')

In [ ]:
# 각 발 전체 인원 중 유망주 비율
print(train[(train['PreferredFoot'] == 'Left')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['PreferredFoot'] == 'Left']['ID'].count(),
      train[(train['PreferredFoot'] == 'Right')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['PreferredFoot'] == 'Right']['ID'].count(),
)
# 전체 인원 중 각 발의 비율
print(train[train['PreferredFoot'] == 'Left']['ID'].count() / train['ID'].count(),
      train[train['PreferredFoot'] == 'Right']['ID'].count() / train['ID'].count(),
)
# 유망주 전체 인원 중 각 발의 비율
print(train[(train['PreferredFoot'] == 'Left')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['Prospect'] == 1]['ID'].count(),
      train[(train['PreferredFoot'] == 'Right')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['Prospect'] == 1]['ID'].count(),
)

1. 각 발에 대해 전체 인원 중 유망주 비율
- 왼발 : $0.33$
- 오른발 : $0.37$

2. 전체 인원과 각 발의 비율
- 왼발 : $0.24$
- 오른발 : $0.75$

3. 유망주 전체 인원과 각 발의 비율
- 왼발 : $0.22$
- 오른발 : $0.78$

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (10, 10))

for i, val in enumerate(['AttackingWorkRate', 'DefensiveWorkRate']):
  if i == 0:
    color = np.array(sns.color_palette('OrRd'))[[1, 3, 5], :]
  elif i == 1:
    color = np.array(sns.color_palette('Blues'))[[1, 3, 5], :]
  
  (train[val].value_counts()[['Low', 'Medium', 'High']]
                          .to_frame()
                          .T
                          .plot(kind = 'bar', 
                                stacked = True, 
                                rot = 0,
                                ax = ax[i][0],
                                color = color)
                          )
  ax[i][0].legend().set_visible(False)

  (train[train['Prospect'] == 1][val].value_counts()[['Low', 'Medium', 'High']]
                                                  .to_frame()
                                                  .T
                                                  .plot(kind = 'bar', 
                                                        stacked = True, 
                                                        rot = 0,
                                                        ax = ax[i][1],
                                                        color = color
  ))

  ax[i][1].set_ylim(ax[i][0].get_ylim())

In [ ]:
# 위 사항은 수치적으로 보고 싶음 : WorkRate에는 총 9가지 유형이 있는데, 이에 따른 생존율을 비교해보자
temp = (train[['ID', 'AttackingWorkRate', 'DefensiveWorkRate', 'Prospect']]
                                                              .groupby(['AttackingWorkRate', 'DefensiveWorkRate', 'Prospect'])['ID']
                                                              .count()
                                                              .to_frame()
                                                              .reset_index().rename(columns = {"ID" : 'counts'})
)
temp.head(2)

In [ ]:
sns.catplot(data = temp, 
            x = 'AttackingWorkRate', 
            y = 'counts', 
            hue = 'DefensiveWorkRate', 
            col = 'Prospect', 
            kind = 'bar', 
            order = ['Low', 'Medium', 'High'], 
            hue_order = ['Low', 'Medium', 'High'],
            ax = ax)
plt.suptitle("ATT / DEF WorkRate and The Num of Prospect", y = 1.05, fontsize = 15)
plt.tight_layout()

In [ ]:
temp['rates'] = temp['counts'] / temp.groupby(['AttackingWorkRate', 'DefensiveWorkRate'])['counts'].transform("sum")
temp_hm = temp[temp['Prospect'] == 1][:]
temp_hm = temp_hm.pivot(index = 'AttackingWorkRate', columns = 'DefensiveWorkRate', values = 'rates').loc[['High', 'Medium', 'Low'], ['Low', 'Medium', 'High']]

In [ ]:
plt.figure(figsize = (7, 7))
sns.heatmap(data = temp_hm, annot = True, fmt = '.3f', cmap = 'Blues', vmin = 0, vmax = 1)
plt.title("Att / Def WorkRate and Prospect Rate")

- `Low, Low`에 있는 요소는 샘플이 1개니까 크게 고려할 요소는 아님
- 애초에 예상한 건 `오른쪽 위로 갈수록 활동량이 좋으니 Prospect 비율이 높을 것이다` 였는데, 그렇지는 않은 것으로 나타남

#### 2. `Position`

In [ ]:
train['Position'].unique()

In [ ]:
# 먼저 시각화 하고 시작함
fig, ax = plt.subplots(figsize = (10, 5))
position_order = ['GK', 'LB', 'CB', 'RB', 'LWB', 'CDM', 'RWB', 'LM', 'CM', 'RM', 'CAM', 'LW', 'CF', 'RW', 'ST']
(train[['Position', 'Prospect']].value_counts()
                                .to_frame()
                                .reset_index()
                                .rename(columns = {0 : 'counts'})
                                .pivot(index = 'Position', 
                                       columns = 'Prospect', 
                                       values = 'counts')
                                .loc[position_order]
                                .plot(kind = 'bar', stacked = True, ax = ax)
)
plt.title("Position and Prospect")
plt.gca().set_axisbelow(True)
plt.grid(True, axis = 'y')

- CAM, CB, ST, GK, RM 순으로 데이터의 양이 많음
- 

### 양적 데이터
1. 

In [ ]:
plt.figure(figsize = (7, 7))

corr = train.corr()
mask = np.zeros_like(corr, dtype = bool)
mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것

sns.heatmap(data = corr, cmap = 'coolwarm', mask = mask) 

- 위 히트맵에서, 일부 필요없는 부분을 제외하고 시각화 함
- 보고 싶은 건 2가지임
  1. 각종 스탯 간의 관계
  2. 각종 스탯과 `Position Rating`의 관계
- 또한, 

In [ ]:
border = train.corr().columns.get_loc('GKReflexes') + 1 # 47

In [ ]:
# 1. 스탯 간의 관계 보기

# 근데 그냥 0 이상으로 두는게 낫지 않을까 싶으요?
# 요건 솔직히 어떻게 해놔야 될지 모르겠다. 다음에 보도록 하자.
corr1 = train.corr().iloc[:border, :border]
mask = np.zeros_like(corr1, dtype = bool)
mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것


plt.figure(figsize = (12, 12))
sns.heatmap(data = corr1, 
            cmap = 'PRGn', 
            mask = (mask 
                   # | np.where(corr1 > 0.5, 0, 1)
                    ), 
            vmin = -1, 
            vmax = 1) 

- 전체적으로 봤을 때, 모든 값을 모델에 포함한다면 중복되는 영역이 생김
  - 예를 들면 ~~Total에 들어가는 값에는 중복이 있을 것
  - 또한 `Rating`에 대한 수치도 중복이 있을 것이다

In [ ]:
corr2 = train.corr().iloc[:border, border:]
# mask = np.zeros_like(corr2, dtype = bool)
# mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것

plt.figure(figsize = (12, 12))
sns.heatmap(data = corr2, 
            cmap = 'PRGn', 
            # mask = np.where(corr2 > 0.3, 0, 1),
            vmin = -1, 
            vmax = 1) 

- 이렇게 놓고 봤을 때 명백히 나뉘는 구역이 3가지가 있음
  - `LM, CM, RM` 영역 및 이보다 공격적인 위치
  - `LWB, CDM, RWB` 영역 ~ 수비수 영역까지 수비적인 위치
  - `GK`
- 이렇게 나눠지는 영역에서도 `CM`이나 `CDM`은 다른 경향을 보임


In [ ]:
foot_order = ['Left', 'Right']
work_rate_order = ['Low', 'Medium', 'High']

def column_label_order(df, col, order_lst):

  LE = LabelEncoder()
  LE.fit(df[col])
  LE.classes_ = np.array(order_lst)
  df[col] = LE.transform(df[col])


column_label_order(train, 'PreferredFoot', foot_order)
column_label_order(train, 'AttackingWorkRate', work_rate_order)
column_label_order(train, 'DefensiveWorkRate', work_rate_order)

In [ ]:
train.columns

In [ ]:
train_ab = train.iloc[:, np.r_[0: train.columns.get_loc('DefensiveWorkRate') + 1, train.columns.get_loc('Crossing') : train.columns.get_loc('GKReflexes') + 1, -1]]
train_ab.head(2)

In [ ]:
train_ab_avg = (train_ab.iloc[:, 1:].groupby(['Position', 'Prospect'])
                                    .mean()
                                    .reset_index()
                                    .set_index(['Position', 'Prospect'])
                                    .T
                )
train_ab_avg

- 위 표를 보면 가장 의외인 지점은, 같은 포지션에서도 `Prospect = 0`의 스탯이 더 높은 수치를 나타내는 곳이 많다.
- `Age`는 `Prospect`를 결정하는 중요한 요인이라고 생각됨.

In [ ]:
sns.countplot(data = train, x = 'Age', hue = 'Prospect')

결론)
1. `Position`을 분리할 건데,
  1. `Left, Center, Right, GK`라는 횡적인 위치와
  2. `ATT, MID, DEF, GK`라는 종적인 위치 2가지로 나누겠음

  - 이렇게 할 경우 CM과 CDM의 차이가 모호해진다는 문제점이 생길 수는 있는데, 일단 그렇게 진행하겠음

2. 중복된 인풋을 제거하기 위해, 모델에 활용하는 수치는 `~~Total` 값과 `~~Rating`을 제외한 모든 값으로 하겠음

In [ ]:
train['Position'].unique()

In [ ]:
def change_position(df, col_name):
  if df[col_name] == 'CB':
    df['hor_pos'] = 'center'
    df['ver_pos'] = 'DEF'
  elif df[col_name] == 'LB':
    df['hor_pos'] = 'left'
    df['ver_pos'] = 'DEF'
  elif df[col_name] == 'RB':
    df['hor_pos'] = 'right'
    df['ver_pos'] = 'DEF'
  if df[col_name] == 'GK':
    df['hor_pos'] = 'GK'
    df['ver_pos'] = 'GK'
  if df[col_name] == 'LWB':
    df['hor_pos'] = 'center'
    df['ver_pos'] = 'DEF'
  if df[col_name] == 'CB':
    df['hor_pos'] = 'center'
    df['ver_pos'] = 'DEF'

# 전처리
- 나중에 테스트 세트에도 적용해야 하니까 애껴둡시다.

In [ ]:
# 이거는 모델에 넣기 전에 진행하면 됨 - 먼저 나눠버리면 X축이 뭘 의미하는지 모르게 됨

# foot_order = ['Left', 'Right']
# work_rate_order = ['Low', 'Medium', 'High']
# position_order = ['GK', 'LB', 'CB', 'RB', 'LWB', 'CDM', 'RWB', 'LM', 'CM', 'RM', 'CAM', 'LW', 'CF', 'RW', 'ST']

# def column_label_order(df, col, order_lst):

#   LE = LabelEncoder()
#   LE.fit(df[col])
#   LE.classes_ = np.array(order_lst)
#   df[col] = LE.transform(df[col])


# column_label_order(train, 'PreferredFoot', foot_order)
# column_label_order(train, 'AttackingWorkRate', work_rate_order)
# column_label_order(train, 'DefensiveWorkRate', work_rate_order)
# column_label_order(train, 'Position', position_order)


In [ ]:
test.columns

#### 트리 모델을 쓰겠다면 OrdinalEncoder도 고려해보자
- 원핫인코딩은 트리 모델에서 그렇게 유용하지 않은 지표일 수 있다
  - 원핫인코딩 특성이 선택되었을 때, 차원은 증가하는데, 분류 기준으로서의 가치가 낮아지기 떄문임

In [ ]:
# 범주형 데이터들 원핫인코딩
train = pd.read_csv('/content/drive/MyDrive/data/train.csv', index_col = 'ID')
train_X = train.iloc[:, np.r_[:3, train.columns.get_loc('Crossing'):train.columns.get_loc('GKReflexes') + 1]]
train_y = train['Prospect']


for i in ['PreferredFoot', 'AttackingWorkRate', 'DefensiveWorkRate', 'Position']:
  onehot = pd.get_dummies(train[i], 
                #  drop_first = True,
                prefix = i
                )
  train_X = train_X.join(onehot)

In [ ]:
# 나머지 모든 데이터를 0과 1 사이로 표준화함
ss = StandardScaler()
ss.fit(train_X.iloc[:, :train_X.columns.get_loc('GKReflexes') + 1])
train_X.iloc[:, :train_X.columns.get_loc('GKReflexes') + 1] = ss.transform(train_X.iloc[:, :train_X.columns.get_loc('GKReflexes') + 1])

In [ ]:
# test에 대해서도 동일한 과정을 적용한다
test_df = pd.read_csv('/content/drive/MyDrive/data/test.csv', index_col = 'ID')
test = test_df.iloc[:, np.r_[:3, train.columns.get_loc('Crossing'):train.columns.get_loc('GKReflexes') + 1]]

for i in ['PreferredFoot', 'AttackingWorkRate', 'DefensiveWorkRate', 'Position']:
  onehot = pd.get_dummies(test_df[i], 
                #  drop_first = True,
                prefix = i
                )
  test = test.join(onehot)

# 훈련 세트에 적용한 스케일러를 테스트 세트에도 동일하게 적용한다
test.iloc[:, :test.columns.get_loc('GKReflexes') + 1] = ss.transform(test.iloc[:, :test.columns.get_loc('GKReflexes') + 1])
test

# 모델 적용

## 1. 랜덤 포레스트

In [ ]:
# sub_X, val_X, sub_y, val_y = train_test_split(train_X, train_y, test_size = 0.25, random_state = 0, stratify = train_y)

In [ ]:
# clf = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 0, bootstrap = True,oob_score = True)
# clf.fit(sub_X, sub_y)
# pred_y = clf.predict(sub_X)
# print("Train Set Accuracy : ", clf.score(sub_X, sub_y))
# print("f1 score : ", f1_score(sub_y, pred_y))

In [ ]:
# print("Validation Set Accuracy : ", clf.score(val_X, val_y))
# print("f1_score() : " , f1_score(val_y, clf.predict(val_X)))

In [ ]:
# # 교차검증을 보자
# skf = StratifiedKFold(n_splits = 10, shuffle  = True, random_state = 0)
# skf.get_n_splits(train_X, train_y) 

# clf = RandomForestClassifier(n_estimators = 100, max_depth = 5, bootstrap = True, oob_score = True, random_state = 0)
# cross_val_score(clf, train_X, train_y, cv = skf, scoring = 'accuracy').mean()

In [ ]:
# 하이퍼 파라미터 튜닝도 해보자
params = {'max_depth' : list(range(5, 10)),
          'n_estimators' : list(range(50, 1000)),
          'bootstrap' : [True, False],
          'max_features' : ['sqrt', None]
          }
clf = RandomForestClassifier(random_state = 0)

model = RandomizedSearchCV(estimator = clf,
                           param_distributions = params,
                           n_iter = 30,
                           cv = 5,
                           n_jobs = -1, 
                           verbose = 1)

model.fit(train_X, train_y)

In [ ]:
model.best_params_

In [ ]:
model.score(train_X, train_y)

In [ ]:
pred = model.predict(test)
pred

In [ ]:
submit = pd.DataFrame({'Prospect' : pred}, index = test.index)
submit
submit.to_csv('submit_RF2.csv')

In [ ]:
# Ordinial Encoder를 써본다든가
# Age Height Weight를 써본다든가
# 뭐 등등이 있겠쥬?

## 2. DNN
- 가장 간단한 타입을 써보겠읍니다